In [0]:
!pip install nflgame-redux==2.0.1b1

     |████████████████████████████████| 44.1MB 51kB/s 


In [0]:
!nflgame-update-schedule

Updating (2019, 'REG', 12)
Last updated: 2019-11-23 20:12:35.485941


In [0]:
import nflgame

In [0]:
games=nflgame.games(2017,week=2)

In [0]:
plays=nflgame.combine_plays(games)

In [0]:
for play in plays.sort('kicking_fga'):
  print(play)

In [0]:
games=nflgame.games(2019,week=10)
kick_l=[]
for game in games:
  weather=get_weather_dict(2019,10,game.home,game.away)
  for drive in game.drives:
    if drive.result=="Field Goal" or drive.result=="Missed FG":
      print("\n\n")
      for play in drive.plays:
        kick={}
        if len(play.events)>0:
          if play.events[0].get('kicking_fga',0):
            print(play.data)
            print(play.events)
            if play.events[0].get('kicking_fgm',0):
              kick['Total Yards']=play.events[0]['kicking_fgm_yds']
              kick['Made']=1
            else:
              kick['Total Yards']=play.events[0]['kicking_fgmissed_yds']
              kick['Made']=0

            kick['Quarter']=play.data['qtr']
            kick['Time']=play.data['time']
            kick['Down']=play.data['down']
            kick['Kicker id']=play.events[0]['playerid']
            kick['Kicker Name']=play.events[0]['playername']
            kick['Kicking Team']=play.data['posteam']

            q_weather=get_weather_by_quarter(weather,play.data['qtr'])
            kick['Weather_Condition']=q_weather['Condition']
            kick['Humidity']=int(q_weather['Humidity'][:-1])
            kick['Temperature']=int(q_weather['Temperature'])
            kick['Wind']=int(q_weather['Wind'][:-2])
            kick['Stadium']=weather['Stadium']['Stadium Name']
            kick['Grass Type']=weather['Stadium']['Stadium Surface']
            kick['Stadium Type']=weather['Stadium']['Stadium Type']
            kick_l.append(kick)


In [0]:
kick_l

In [0]:
def get_weather_by_quarter(weather,quarter):
  quarter_dict={1:"Kickoff",2:"Q2",3:"Q3",4:"Q4",5:"Q4"}
  try:
    w=weather[quarter_dict[quarter]]
  except:
    w=weather.get(quarter_dict[1],0)
    if not w:
      w={'Barometer': '',
  'Condition': '',
  'Humidity': '',
  'Temperature': '',
  'Wind': ''}
  return w

In [0]:
all_kicks=[]
for season in range(2018,2019):
  for week in range(1,18):
    print(season,week)
    try:
      games=nflgame.games(season,week=week)
    except:
      games=[]
    for game in games:
      weather=get_weather_dict(season,week,game.home,game.away)
      if weather:
        for drive in game.drives:
          if drive.result=="Field Goal" or drive.result=="Missed FG":
            for play in drive.plays:
              kick={}
              if len(play.events)>0:
                if play.events[0].get('kicking_fga',0):
                  if play.events[0].get('kicking_fgm',0):
                    kick['Total Yards']=play.events[0]['kicking_fgm_yds']
                    kick['Made']=1
                  else:
                    kick['Total Yards']=play.events[0]['kicking_fgmissed_yds']
                    kick['Made']=0
                  kick['Quarter']=play.data['qtr']
                  kick['Time']=play.data['time']
                  kick['Down']=play.data['down']
                  kick['Kicker id']=play.events[0]['playerid']
                  kick['Kicker Name']=play.events[0]['playername']
                  kick['Kicking Team']=play.data['posteam']
                  q_weather=get_weather_by_quarter(weather,play.data['qtr'])
                  kick['Weather_Condition']=q_weather['Condition']
                  kick['Humidity']=q_weather['Humidity'][:-1]
                  kick['Temperature']=q_weather['Temperature']
                  kick['Wind']=q_weather['Wind'][:-2]
                  kick['Stadium']=weather['Stadium']['Stadium Name']
                  kick['Grass Type']=weather['Stadium']['Stadium Surface']
                  kick['Stadium Type']=weather['Stadium']['Stadium Type']

                  all_kicks.append(kick)

2018 1
http://www.nflweather.com/en/game/2018/week-1/Falcons-at-Eagles
Lincoln Financial Field
http://www.nflweather.com/en/game/2018/week-1/Bills-at-Ravens
M&T Bank Stadium
http://www.nflweather.com/en/game/2018/week-1/Steelers-at-Browns
FirstEnergy Stadium
http://www.nflweather.com/en/game/2018/week-1/Bengals-at-Colts
Lucas Oil Stadium
http://www.nflweather.com/en/game/2018/week-1/Titans-at-Dolphins
Hard Rock Stadium
http://www.nflweather.com/en/game/2018/week-1/49ers-at-Vikings
U.S. Bank Stadium
http://www.nflweather.com/en/game/2018/week-1/Texans-at-Patriots
Gillette Stadium
http://www.nflweather.com/en/game/2018/week-1/Buccaneers-at-Saints
Mercedes-Benz Superdome
http://www.nflweather.com/en/game/2018/week-1/Jaguars-at-Giants
MetLife Stadium
http://www.nflweather.com/en/game/2018/week-1/Chiefs-at-Chargers
ROKiT Field at Dignity Health Sports Park
http://www.nflweather.com/en/game/2018/week-1/Redskins-at-Cardinals
State Farm Stadium
http://www.nflweather.com/en/game/2018/week-1/Cow

In [0]:
def get_weather_dict(season,week,home_team,away_team):
  NFLWeather="http://www.nflweather.com/en/game/{}/week-{}/{}"
  teams=team_format(home_team,away_team)
  print(NFLWeather.format(season,week,teams))
  page=requests.get(NFLWeather.format(season,week,teams))
  if page.text:
    doc=pq(page.text)
    weather_q={}
    for span in doc('.span3'):
      temp_reading={}
      ss=pq(span)
      quarter=ss.find('b').eq(0).text()
      if quarter:
        conditions=ss.find('p').eq(3).text()
        tmp=ss("p:Contains('Temperature:')")('b').text()
        hum=ss("p:Contains(' Humidity: ')")('b').text()
        bar=ss("p:Contains(' Barometer: ')")('b').text()
        wind=ss("p:Contains(' Wind: ')")('b').text().split(" ")[0]
        temp_reading['Condition']=conditions
        temp_reading['Temperature']=tmp[:-2]
        temp_reading['Humidity']=hum
        temp_reading['Barometer']=bar
        temp_reading['Wind']=wind
        weather_q[quarter]=temp_reading

    stadium=doc('.span4')
    std={}
    std_doc=pq(stadium)
    print(std_doc.find('b').eq(0).text())
    std['Stadium Name']=std_doc.find('b').eq(0).text()
    std['Stadium Surface']=std_doc.find('b').eq(1).text()
    std['Stadium Type']=std_doc.find('b').eq(5).text()

    weather_q['Stadium']=std

    return weather_q
  else:
    return None


In [0]:
import pandas

df=pandas.DataFrame(all_kicks)

In [0]:
df2=pandas.DataFrame(all_kicks)

In [0]:
df2

,Total Yards,Made,Quarter,Time,Down,Kicker id,Kicker Name,Kicking Team,Weather_Condition,Humidity,Temperature,Wind,Stadium,Grass Type,Stadium Type
0,21,1,1,05:03,4,00-0020578,M.Bryant,ATL,Rain,70,81,2,Lincoln Financial Field,Desso GrassMaster,Open
1,26,1,2,06:12,4,00-0033787,J.Elliott,PHI,Partly Cloudy,75,79,2,Lincoln Financial Field,Desso GrassMaster,Open
2,52,1,2,02:13,4,00-0020578,M.Bryant,ATL,Partly Cloudy,75,79,2,Lincoln Financial Field,Desso GrassMaster,Open
3,41,1,2,13:18,4,00-0029597,J.Tucker,BAL,Light Rain,92,63,12,M&T Bank Stadium,419 Tifway Bermuda Grass,Open
4,52,0,2,08:48,4,00-0025944,S.Hauschka,BUF,Light Rain,92,63,12,M&T Bank Stadium,419 Tifway Bermuda Grass,Open
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,36,1,1,07:25,4,00-0033862,Z.Gonzalez,ARI,Mostly Cloudy,77,45,3,CenturyLink Field,Fieldturf,Open
871,50,1,2,01:18,4,00-0033862,Z.Gonzalez,ARI,Mostly Cloudy,76,46,2,CenturyLink Field,Fieldturf,Open
872,42,1,4,06:25,4,00-0019646,S.Janikowski,SEA,Mostly Cloudy,73,46,0,CenturyLink Field,Fieldturf,Open
873,55,1,4,01:54,4,00-0033862,Z.Gonzalez,ARI,Mostly Cloudy,73,46,0,CenturyLink Field,Fieldturf,Open


In [0]:
df=df.append(df2)

In [0]:
df.to_csv("kicks_(2009-2018[not 2010]).csv")

In [0]:
from google.colab import files
files.download("kicks_(2009-2018[not 2010]).csv")

# Weather

In [0]:
NFLWeather="http://www.nflweather.com/en/game/{}/week-{}/{}"

In [0]:
!pip install pyquery

In [0]:
from pyquery import PyQuery as pq
import requests

In [0]:
NFLWeather.format(2011,6,'jaguars-at-steelers')

'http://www.nflweather.com/en/game/2011/week-6/jaguars-at-steelers'

In [0]:
page=requests.get(NFLWeather.format(2011,6,'jaguars-at-steelers'))

In [0]:
print(page.text)

In [0]:
doc=pq(page.text)

In [0]:
weather_q={}
for span in doc('.span3'):
  temp_reading={}
  ss=pq(span)
  quarter=ss.find('b').eq(0).text()
  if quarter:
    conditions=ss.find('p').eq(3).text()
    tmp=ss("p:Contains('Temperature:')")('b').text()
    hum=ss("p:Contains(' Humidity: ')")('b').text()
    bar=ss("p:Contains(' Barometer: ')")('b').text()
    wind=ss("p:Contains(' Wind: ')")('b').text().split(" ")[0]
    temp_reading['Condition']=conditions
    temp_reading['Temperature']=tmp[:-2]
    temp_reading['Humidity']=hum
    temp_reading['Barometer']=bar
    temp_reading['Wind']=wind
    weather_q[quarter]=temp_reading


In [0]:
weather_q

{'Kickoff': {'Barometer': '29"',
  'Condition': 'Mostly Cloudy',
  'Humidity': '38%',
  'Temperature': '64',
  'Wind': ''}}

In [0]:
get_weather_dict(2019,2,'ATL','PHI')

http://www.nflweather.com/en/game/2019/week-2/Eagles-at-Falcons
Mercedes-Benz Stadium


{'Kickoff': {'Barometer': '1017"',
  'Condition': 'Partly Cloudy',
  'Humidity': '56%',
  'Temperature': '84',
  'Wind': '2mi'},
 'Q2': {'Barometer': '1017"',
  'Condition': 'Partly Cloudy',
  'Humidity': '61%',
  'Temperature': '82',
  'Wind': '2mi'},
 'Q3': {'Barometer': '1017"',
  'Condition': 'Partly Cloudy',
  'Humidity': '65%',
  'Temperature': '80',
  'Wind': '2mi'},
 'Q4': {'Barometer': '1017"',
  'Condition': 'Partly Cloudy',
  'Humidity': '68%',
  'Temperature': '79',
  'Wind': '2mi'},
 'Stadium': {'Stadium Name': 'Mercedes-Benz Stadium',
  'Stadium Surface': 'FieldTurf Revolution 360',
  'Stadium Type': 'Dome'}}

In [0]:
def team_format(home_team,away_team):
  team_list_dict={'ARI': 'Cardinals',
 'ATL': 'Falcons',
 'BAL': 'Ravens',
 'BUF': 'Bills',
 'CAR': 'Panthers',
 'CHI': 'Bears',
 'CIN': 'Bengals',
 'CLE': 'Browns',
 'DAL': 'Cowboys',
 'DEN': 'Broncos',
 'DET': 'Lions',
 'GB': 'Packers',
 'HOU': 'Texans',
 'IND': 'Colts',
 'JAC': 'Jaguars',
 'JAX':'Jaguars',
 'KC': 'Chiefs',
 'LAC': 'Chargers',
 'LA': 'Rams',
 'SD': 'Chargers',
 'MIN': 'Vikings',
 'MIA': 'Dolphins',
 'NE': 'Patriots',
 'NO': 'Saints',
 'NYG': 'Giants',
 'NYJ': 'Jets',
 'OAK': 'Raiders',
 'PHI': 'Eagles',
 'PIT': 'Steelers',
 'SEA': 'Seahawks',
 'SF': '49ers',
 'STL': 'Rams',
 'TB': 'Buccaneers',
 'TEN': 'Titans',
 'WAS': 'Redskins'}
  home=team_list_dict[home_team]
  away=team_list_dict[away_team]
  return "{}-at-{}".format(away,home)

In [0]:
team_format("ATL","PHI")